## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [11]:
import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt
from matplotlib import ticker
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn-notebook')# Your code here - remember to use markdown cells for comments as well!

In [12]:
kc_house_df = pd.read_csv('./data/kc_house_data.csv', parse_dates=['date']) # specify index_col=0 to avoid creating an "Unnamed: 0" column.

# specify parse date to avoid object dtype for dates

kc_house_df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,2014-05-21,360000.0,3,2.50,1530,1131,3.0,NO,NONE,...,8 Good,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
21593,6600060120,2015-02-23,400000.0,4,2.50,2310,5813,2.0,NO,NONE,...,8 Good,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200
21594,1523300141,2014-06-23,402101.0,2,0.75,1020,1350,2.0,NO,NONE,...,7 Average,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007
21595,291310100,2015-01-16,400000.0,3,2.50,1600,2388,2.0,NaN,NONE,...,8 Good,1600,0.0,2004,0.0,98027,47.5345,-122.069,1410,1287


In [13]:
kc_house_df.drop_duplicates(subset='id', inplace=True)

In [14]:
kc_house_df['id'].value_counts()

2911700010    1
5450300010    1
5104511600    1
1160000115    1
686530110     1
             ..
2115510470    1
2922701305    1
6071600370    1
526059224     1
1777500160    1
Name: id, Length: 21420, dtype: int64

In [15]:
kc_house_df.isnull().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2353
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3804
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [16]:
kc_house_df['waterfront'].fillna('no', inplace=True)
kc_house_df['yr_renovated'].fillna(0, inplace=True)
kc_house_df['view'].fillna('NONE', inplace=True)

In [17]:
kc_house_df.isnull().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [18]:
kc_house_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,no,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,0.0,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [19]:
kc_house_df.drop(columns= ['lat', 'long'], inplace=True)

In [20]:
kc_house_df.corr()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15
id,1.000000,-0.018525,0.000691,0.004343,-0.013451,-0.132648,0.018139,-0.011794,0.021171,-0.010857,-0.007934,-0.003883,-0.139429
price,-0.018525,1.000000,0.309453,0.526229,0.701875,0.089111,0.256286,0.605294,0.052906,0.118484,-0.052371,0.584549,0.082438
bedrooms,0.000691,0.309453,1.000000,0.515383,0.579069,0.032490,0.177734,0.480242,0.155241,0.017470,-0.155061,0.394949,0.030570
bathrooms,0.004343,0.526229,0.515383,1.000000,0.755522,0.087779,0.501803,0.686328,0.506252,0.047686,-0.203884,0.569453,0.087760
sqft_living,-0.013451,0.701875,0.579069,0.755522,1.000000,0.172586,0.352868,0.876533,0.316646,0.050851,-0.198995,0.756186,0.183837
sqft_lot,-0.132648,0.089111,0.032490,0.087779,0.172586,1.000000,-0.005561,0.183210,0.052312,0.004991,-0.129422,0.143805,0.717743
floors,0.018139,0.256286,0.177734,0.501803,0.352868,-0.005561,1.000000,0.522751,0.488935,0.003141,-0.058532,0.279071,-0.011367
sqft_above,-0.011794,0.605294,0.480242,0.686328,0.876533,0.183210,0.522751,1.000000,0.422977,0.020078,-0.260803,0.731887,0.194445
yr_built,0.021171,0.052906,0.155241,0.506252,0.316646,0.052312,0.488935,0.422977,1.000000,-0.203147,-0.346162,0.324715,0.070125
yr_renovated,-0.010857,0.118484,0.017470,0.047686,0.050851,0.004991,0.003141,0.020078,-0.203147,1.000000,0.062291,0.000812,0.004317


In [21]:
reno_filter = kc_house_df.loc[kc_house_df['yr_renovated'] > 2000 ]
reno_filter

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15
35,9547205180,2014-06-13,696000.0,3,2.50,2300,3060,1.5,NO,NONE,Average,8 Good,1510,790.0,1930,2002.0,98115,1590,3264
103,2450000295,2014-10-07,1090000.0,3,2.50,2920,8113,2.0,NO,NONE,Average,8 Good,2920,0.0,1950,2010.0,98004,2370,8113
158,8029200135,2014-11-13,247000.0,3,2.00,1270,7198,1.5,NO,NONE,Average,7 Average,1270,0.0,1916,2013.0,98022,1160,7198
237,7228500560,2015-03-20,410000.0,4,1.00,1970,4740,1.5,NO,NONE,Average,7 Average,1670,300.0,1904,2005.0,98122,1510,4740
274,4235400186,2014-11-24,331000.0,3,1.75,1080,1306,1.0,NO,NONE,Average,7 Average,580,500.0,1954,2003.0,98199,1440,2225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19602,6392000625,2014-07-12,451000.0,2,1.00,900,6000,1.0,NO,NONE,Average,7 Average,900,0.0,1944,2004.0,98115,1460,4800
20041,126039256,2014-09-04,434900.0,3,2.00,1520,5040,2.0,NO,NONE,Average,7 Average,1520,0.0,1977,2006.0,98177,1860,8710
20428,4305600360,2015-02-25,500012.0,4,2.50,2400,9612,1.0,NO,NONE,Average,8 Good,1230,1170.0,1962,2009.0,98059,2430,5539
20431,3319500628,2015-02-12,356999.0,3,1.50,1010,1546,2.0,no,NONE,Average,8 Good,1010,0.0,1971,2014.0,98144,1010,1517
